In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:43677")
client

<Client: 'tcp://127.0.0.1:43677' processes=8 threads=32, memory=125.88 GiB>

In [2]:
import dask.bag as db
import dask.dataframe as df
import json

In [50]:
filename = f"quotes-{2020}.json"
dataset = db.read_text(f"../quotebank/raw/{filename}", blocksize=100e6).map(json.loads)

In [18]:
def remove_probability_of_others_then_most(quote):
    del quote["probas"]
    return quote

In [19]:
def extract_probability_of_most_prob_speaker(quote):
    quote["speaker_prob"] = float(quote["probas"][0][1])
    return quote

In [20]:
def extract_only_one_qid(quote):
    quote["qids"] = quote["qids"] if len(quote["qids"]) == 1 else None
    return quote

In [21]:
import csv

In [29]:
domains_by_country_dict = {}

with open('datasets/GDELTDOMAINSBYCOUNTRY-MAY2018.TXT') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    next(reader, None) # skip header
    domains_by_country_dict = {rows[0]:rows[1] for rows in reader}
    

In [30]:
import tld

In [51]:
def convert_domain_to_country(quote, domains_by_country_dict):
    #add countries list to data based on the GDELT dataset
    countries_set = set()
    for url in quote["urls"]:
        domain_name = tld.get_fld(url)
        if domain_name in domains_by_country_dict:
            countries_set.add(domains_by_country_dict[domain_name])
    quote["url_countries"] = list(countries_set) if len(countries_set) != 0  else None
    return quote


In [58]:
dataset1 = dataset.map(extract_probability_of_most_prob_speaker)
dataset2 = dataset1.map(remove_probability_of_others_then_most)
dataset3 = dataset2.map(extract_only_one_qid)
dataset4 = dataset3.map(convert_domain_to_country, domains_by_country_dict)

In [64]:
dataset4.filter(lambda quote: len(quote["url_countries"])>1 if quote["url_countries"] else 0 ).take(10)

({'quoteID': '2020-04-08-001814',
  'quotation': 'After 10 years of growth we will experience a recession this year,',
  'speaker': 'Peter Altmaier',
  'qids': ['Q65539'],
  'date': '2020-04-08 07:54:20',
  'numOccurrences': 5,
  'urls': ['http://www.breitbart.com/news/france-germany-face-historic-economic-declines/',
   'https://www.ibtimes.com/german-economy-shrink-nearly-10-q2-experts-2954923',
   'https://www.financialexpress.com/economy/europes-biggest-german-economy-to-shrink-by-nearly-10-in-q2-say-experts/1922698/',
   'http://www.courthousenews.com/german-economy-takes-a-staggering-hit/',
   'https://www.business-standard.com/article/pti-stories/france-germany-face-historic-economic-declines-120040801280_1.html'],
  'phase': 'E',
  'speaker_prob': 0.7357,
  'url_countries': ['US', 'IN']},
 {'quoteID': '2020-02-16-002091',
  'quotation': 'an expectation of the contact Aboriginal and Torres Strait Islander people and particularly young people will have with police',
  'speaker': 